In [15]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from uuid import uuid4
from dotenv import load_dotenv
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
load_dotenv()

True

In [2]:
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_DEPLOYMENT_ID = os.getenv('AZURE_OPENAI_DEPLOYMENT_ID')
AZURE_OPENAI_KEY = os.getenv('AZURE_OPENAI_KEY')
AZURE_API_VERSION = os.getenv('AZURE_API_VERSION')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=PINECONE_API_KEY)

In [3]:
index_name = "ocbc-hr-gpt"  # change if desired

index = pc.Index(index_name)

In [4]:
llm = AzureChatOpenAI(
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            azure_deployment=AZURE_OPENAI_DEPLOYMENT_ID,
            api_version=AZURE_API_VERSION,
            api_key=AZURE_OPENAI_KEY,
            temperature=0.0,
            verbose=True,
        )

embedding_llm = AzureOpenAIEmbeddings(
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            azure_deployment='embedding-ada-crayon',
            api_key=AZURE_OPENAI_KEY,
            api_version=AZURE_API_VERSION,
        )

In [5]:
vector_store = PineconeVectorStore(index=index, embedding=embedding_llm)
retriever = vector_store.as_retriever()

In [20]:
retriever.invoke("HR")

[Document(metadata={'page': 7.0, 'source': 'c:\\Users\\san\\Downloads\\Dummy - CB Policy.pdf'}, page_content='to receive a pension as per the respective law of his/her country . \n4.3 Responsibility – HR department / Finance department  \n \n● The Manager HR shall keep the records of the employees at the time \nof joining and/or whose employment service has been terminated.'),
 Document(metadata={'page': 8.0, 'source': 'c:\\Users\\san\\Downloads\\Dummy - CB Policy.pdf'}, page_content="9 \n \nOCBC Information Classification: Internal  ● For finalisation of ESB , the Manager HR and Manager Finance sha ll \nhave to be in constant communication regarding the details of \nemployees leaving the organisation or whose employment service \ncontract is being terminated by the organisation.  \n● For finalisation on visa processing initiation and employment process, \nthe Ma nager HR and the Manager Finance shall have to be in constant \ncommunication to discuss the expenses related to the same.  

Failed to batch ingest runs: LangSmithRateLimitError('Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError(\'429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Monthly unique traces usage limit exceeded"}\')')
Failed to batch ingest runs: LangSmithRateLimitError('Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError(\'429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Monthly unique traces usage limit exceeded"}\')')


In [6]:
prompt = PromptTemplate.from_template(QA_PROMPT)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)



rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

def caller(message):
    response = rag_chain.invoke(message)
    return response

In [7]:
retriever.invoke('Scope & Applicability ')

[Document(metadata={'page': 2.0, 'source': 'c:\\Users\\san\\Downloads\\Dummy - CB Policy.pdf'}, page_content='3 \n \nOCBC Information Classification: Internal  Scope & Applicability  4 \nDefinition & Glossary  4 \nPolicy / Process  Error! Bookmark not def ined.  \n4.1 Policy Definition:  Error! Bookmark not defined.  \n4.2. Procedures  Error! Bookmark not defined.  \n4.3. Responsibility – HR department / Finance department  Error! Bookmark not \ndefined.  \nNon-compliance and consequences  1 \nSpecial Circumstances and Exceptions  Error! Bookmark not defined.'),
 Document(metadata={'page': 3.0, 'source': 'c:\\Users\\san\\Downloads\\Dummy - CB Policy.pdf'}, page_content="4 \n \nOCBC Information Classification: Internal   \n1. Objective  \n \nThe objective of the Compensation and Benefits Policy (The Policy) is aimed \ntowards building a strong framework of pay structure for the organization to \ncreate a  competitive work environment for its employees.  \nThe policy aims  to be transpar

Failed to batch ingest runs: LangSmithRateLimitError('Rate limit exceeded for https://api.smith.langchain.com/runs/batch. HTTPError(\'429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Monthly unique traces usage limit exceeded"}\')')


In [8]:
caller('Scope & Applicability ')

"The scope and applicability of the Compensation and Benefits Policy is to define the principles of the organization's C&B structure and serve as a reference for employees and the HR department. The policy applies to all employees working with the organization. The policy aims to provide transparency and communicate compensation plans to employees."

In [24]:
### Contextualize question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
qa_system_prompt = """You are an experienced Legal Assistant who analyzes legal documents.
Your expertise includes extracting facts and integrating information from multiple sources to provide well-supported answers. 

Guidelines:
1. Derive your answer strictly from the provided context. Do not introduce any new information.
2. Ensure complete contextuality: Address all aspects of the query, linking back to specific details in the context.
3. Avoid phrases like "In the context provided" or "According to my knowledge."

Given the guidelines, please answer the question based on the following context.
If you don't know the answer, just say that you don't know. Keep the answer concise.
If the user ask with BAHASA INDONESIA language, please semantically answer the question with BAHASA INDONESIA language.

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [25]:
conversational_rag_chain.invoke(
    {"input": "what is HR policy?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"The HR policy refers to a set of guidelines and rules established by an organization's Human Resources department. It outlines the principles and procedures that govern various aspects of employment, such as recruitment, compensation, benefits, performance management, employee relations, and termination. The HR policy serves as a framework to ensure consistency, fairness, and compliance with legal requirements in managing the organization's workforce."

In [ ]:
def invoke_hr_policy(input_text, session_id="abc123"):
    response = conversational_rag_chain.invoke(
        {"input": input_text},
        config={
            "configurable": {"session_id": session_id}
        }
    )
    return response["answer"]

# Example usage:
result = invoke_hr_policy("what is HR policy?")
print(result)


In [22]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?'), AIMessage(content="I don't know the answer."), HumanMessage(content='what is HR?'), AIMessage(content='HR stands for Human Resources. It is a department within an organization that is responsible for managing and overseeing various aspects related to employees, such as recruitment, training, performance management, compensation, and employee relations.'), HumanMessage(content='what is HR policy?'), AIMessage(content="An HR policy is a set of guidelines and rules established by an organization's Human Resources department. It outlines the principles, procedures, and practices that govern the management of employees within the organization. HR policies cover various areas such as recruitment, compensation, benefits, performance management, employee conduct, and compliance with legal and ethical standards. These policies help ensure consistency, fairness, and compliance in the organization